In [1]:
import pandas as pd
import numpy as np

In [2]:
automated_df = pd.read_csv("reduced_automated_labels.csv")
manual_df = pd.read_csv("reduced_manual_labels.csv")

In [3]:
automated_df

,IN FILE,MANUAL ID,OFFSET,DURATION,Confidence
0,20210816_030000.WAV,Common Poorwill,36,3,0.108588
1,20210816_014000.WAV,California Towhee,18,3,0.843924
2,20210820_145000.WAV,Bewick's Wren,3,3,0.260280
3,20210820_145000.WAV,Bewick's Wren,9,3,0.200800
4,20210820_145000.WAV,Bewick's Wren,15,3,0.803835
...,...,...,...,...,...
177,20210820_020000_OTH1.WAV,California Towhee,42,3,0.923568
178,20210820_020000_OTH1.WAV,California Towhee,45,3,0.387194
179,20210820_020000_OTH1.WAV,California Towhee,48,3,0.811322
180,20210820_020000_OTH1.WAV,California Towhee,51,3,0.825185


In [4]:
manual_df

,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,LAST MOD DATE
0,20210818_023000.WAV,60,57.5400,0.50,24000,0,384000,California Gnatcatcher,350.585,JacobGlennAyers,09/15/2021
1,20210818_023000.WAV,60,1.4700,0.30,24000,0,384000,California Thrasher,350.570,JacobGlennAyers,09/15/2021
2,20210812_005000.WAV,60,5.2700,1.02,24000,0,384000,Glaucus-winged Gull,70.793,JacobGlennAyers,09/15/2021
3,20210821_015000.WAV,60,59.7150,0.11,24000,0,384000,California Towhee,849.662,JacobGlennAyers,09/15/2021
4,20210821_015000.WAV,60,6.8750,0.17,24000,0,384000,California Thrasher,849.609,JacobGlennAyers,09/15/2021
...,...,...,...,...,...,...,...,...,...,...,...
131,20210820_020000_OTH1.WAV,60,48.6400,0.39,24000,0,384000,California Towhee,206.798,arathbone,09/26/2021
132,20210820_020000_OTH1.WAV,60,59.7200,0.27,24000,0,384000,California Towhee,206.803,arathbone,09/26/2021
133,20210820_020000_OTH1.WAV,60,52.3600,0.24,24000,0,384000,California Towhee,206.799,arathbone,09/26/2021
134,20210820_020000_OTH1.WAV,60,54.6700,0.27,24000,0,384000,California Towhee,206.800,arathbone,09/26/2021


In [5]:
# Creating a date column "DATE" for the automated labels
date_list = []
for row in automated_df.index:
    full_string = automated_df["IN FILE"][row]
    temp_list = full_string.split("_")
    #print(temp_list[0])
    year = temp_list[0][0:4]
    month = temp_list[0][4:6]
    day = temp_list[0][6:8]
    date = month+'/'+day+'/'+year
    #print(date)
    date_list.append(date)
automated_df["DATE"] = date_list

In [6]:
# Creating a date column "DATE" for manual labels
date_list = []
for row in manual_df.index:
    full_string = manual_df["IN FILE"][row]
    temp_list = full_string.split("_")
    #print(temp_list[0])
    year = temp_list[0][0:4]
    month = temp_list[0][4:6]
    day = temp_list[0][6:8]
    date = month+'/'+day+'/'+year
    #print(date)
    date_list.append(date)
manual_df["DATE"] = date_list

In [7]:
# Creating the list of species, the lengths of these will be the initial dimensions of
# a numpy array that will serve as the confusion matrix
automated_species_list = automated_df["MANUAL ID"].to_list()
automated_species_list = list(dict.fromkeys(automated_species_list))
manual_species_list = manual_df["MANUAL ID"].to_list()
manual_species_list = list(dict.fromkeys(manual_species_list))
print(len(automated_species_list))
print(len(manual_species_list))

46
16


In [8]:
# Short script that takes the species with the maximum confidence in a given three second interval on an audio clip
# all with respect to the automated labels.
# I need to loop through clip-by-clip, from there, I can get a list of "OFFSET" values, I can create a nested loop
# that takes the MANUAL ID that has the maximum confidence value within said clip with respec to OFFSET.
# This is Mathias' idea to reduce down the number of false positives.

# getting a unique list of clips
automated_clip_list = automated_df["IN FILE"].to_list()
automated_clip_list = list(dict.fromkeys(automated_clip_list))

# Creating an empty dataframe to store the max predictions with respect to time
reduced_automated_df = pd.DataFrame()
# Looping through each audio clip
for clip in automated_clip_list:
    temp_df = automated_df[automated_df["IN FILE"] == clip]
    # getting a unique list of times
    temp_offset_list = temp_df["OFFSET"].to_list()
    temp_offset_list = list(dict.fromkeys(temp_offset_list))
    #print(temp_offset_list)
    # looping through each offset with respect to each clip
    for offset_time in temp_offset_list:
        temp_offset_df = temp_df[temp_df["OFFSET"] == offset_time]
        temp_predictions_list = temp_offset_df["Confidence"].to_list()
        highest_confidence_score = max(temp_predictions_list)
        best_prediction_df = temp_offset_df[temp_offset_df["Confidence"] == highest_confidence_score]
        if reduced_automated_df.empty:
            reduced_automated_df = best_prediction_df
        else:
            reduced_automated_df = reduced_automated_df.append(best_prediction_df)
reduced_automated_df.reset_index(drop=True,inplace=True)


In [9]:
# comment line out if you don't want to reduce the automated dataframe further
automated_df = reduced_automated_df

In [10]:
# breaking down the annotations into day-level labels

# getting a unique list of dates
automated_date_list = automated_df["DATE"].to_list()
automated_date_list =  list(dict.fromkeys(automated_date_list))
manual_date_list = manual_df["DATE"].to_list()
manual_date_list = list(dict.fromkeys(manual_date_list))

In [11]:
automated_date_df = pd.DataFrame()
for date in automated_date_list:
    temp_df = automated_df[automated_df["DATE"] == date]
    
    temp_date_list = temp_df["DATE"].to_list()
    temp_species_list = temp_df["MANUAL ID"].to_list()
    if automated_date_df.empty:
        automated_date_df["DATE"] = temp_date_list
        automated_date_df["SPECIES"] = temp_species_list
    else:
        temp_df = pd.DataFrame()
        temp_df["DATE"] = temp_date_list
        temp_df["SPECIES"] = temp_species_list
        automated_date_df = automated_date_df.append(temp_df)
automated_date_df = automated_date_df.drop_duplicates()
automated_date_df.reset_index(drop=True,inplace=True)
automated_date_df

,DATE,SPECIES
0,08/16/2021,Common Poorwill
1,08/16/2021,California Towhee
2,08/16/2021,California Thrasher
3,08/16/2021,Brown-crested Flycatcher
4,08/16/2021,Wrentit
5,08/16/2021,Savanna Nightjar
6,08/20/2021,Bewick's Wren
7,08/20/2021,Southern Whiteface
8,08/20/2021,Bachman's Sparrow
9,08/20/2021,Small Pratincole


In [12]:
manual_date_df = pd.DataFrame()
for date in manual_date_list:
    temp_df = manual_df[manual_df["DATE"] == date]
    
    temp_date_list = temp_df["DATE"].to_list()
    temp_species_list = temp_df["MANUAL ID"].to_list()
    if manual_date_df.empty:
        manual_date_df["DATE"] = temp_date_list
        manual_date_df["SPECIES"] = temp_species_list
    else:
        temp_df = pd.DataFrame()
        temp_df["DATE"] = temp_date_list
        temp_df["SPECIES"] = temp_species_list
        manual_date_df = manual_date_df.append(temp_df)
manual_date_df = manual_date_df.drop_duplicates()
manual_date_df.reset_index(drop=True,inplace=True)
manual_date_df

,DATE,SPECIES
0,08/18/2021,California Gnatcatcher
1,08/18/2021,California Thrasher
2,08/18/2021,Northern Mockingbird
3,08/18/2021,California Towhee
4,08/12/2021,Glaucus-winged Gull
5,08/21/2021,California Towhee
6,08/21/2021,California Thrasher
7,08/21/2021,No Birds
8,08/16/2021,California Thrasher
9,08/16/2021,California Towhee


In [13]:
# Adding the Confidence levels back into the dataframe
confidence_list = []
for row in automated_date_df.index:
    date = automated_date_df["DATE"][row]
    species = automated_date_df["SPECIES"][row]
    temp_df = automated_df[automated_df["DATE"] == date]
    temp_df = temp_df[temp_df["MANUAL ID"] == species]
    temp_confidence_list = temp_df["Confidence"].to_list()
    temp_confidence = max(temp_confidence_list)
    confidence_list.append(temp_confidence)
automated_date_df["CONFIDENCE"] = confidence_list
automated_date_df

,DATE,SPECIES,CONFIDENCE
0,08/16/2021,Common Poorwill,0.108588
1,08/16/2021,California Towhee,0.843924
2,08/16/2021,California Thrasher,0.852937
3,08/16/2021,Brown-crested Flycatcher,0.464621
4,08/16/2021,Wrentit,0.975010
5,08/16/2021,Savanna Nightjar,0.127671
6,08/20/2021,Bewick's Wren,0.918056
7,08/20/2021,Southern Whiteface,0.130020
8,08/20/2021,Bachman's Sparrow,0.137466
9,08/20/2021,Small Pratincole,0.542817


In [16]:
automated_date_df.to_csv("birdnet-lite_day-level_reduced.csv",index=False)
#manual_date_df.to_csv("volunteer_day-level.csv",index=False)

In [15]:
# had to make some quick edits with excel for the sake of time. I removed the No Birds label and 
# changed Glaucus winged gull to western gull, that was a bad ID on my part
